<a href="https://colab.research.google.com/github/ManikantaMandlem/EE-435-Deep-Learning-course-project/blob/main/DLFAA_project_master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Collate function is not working, start from here 21-02

In [ ]:
import torch
import torchvision
import os
from collections import defaultdict
import numpy as np
import random
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.models as models
from torchsummary import summary
import torch.nn as nn

In [ ]:
params = {
    'batchsize' : 64,
    'shuffle' : True,
    'num_workers' : 8,
    'lr' : 0.0001,
    'epochs' : 1
}

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

threshold = 5

min_images = 2

os.walk(filter for folders more than threshold)

create train, valid1, valid2, test1 and test2

1 - people that are already seen by model

2 - people that are not seen by model

to create 1, move first min_images of a person to train and the rest is assigned randomly with probabilities of 90-5-5 to valid1 and test1

for 2, split the test dataset into to build valid2 and test2
create dataset objects for all 5 and create dataloads for all 5

Download and save resnet model, change the fully connected layer (add fully connected layers if necessary)

Train, validate and test the model....hooray..! :)



In [ ]:
person_dir = []
person_img_path = defaultdict(list)
threshold = 4
# imagesplit_thr = 2
mypath = '/content/drive/MyDrive/DLFAA/Final_Project/lfw'
dirnames = os.listdir(mypath)
dirnames.pop(5000)
for dirname in dirnames:
    path = mypath +'/'+ dirname
    person_img_path[path] = os.listdir(path)

In [ ]:
type1_dict = defaultdict(list)
type2_dict = defaultdict(list)

file1 = open('/content/drive/MyDrive/DLFAA/Final_Project/train.txt', 'r')
file2 = open('/content/drive/MyDrive/DLFAA/Final_Project/test.txt', 'r')

for line in file1:
    type1_dict[mypath +'/' + line.strip()] = person_img_path[mypath +'/' + line.strip()]
for line in file2:
    type2_dict[mypath +'/' + line.strip()] = person_img_path[mypath +'/' + line.strip()]

In [ ]:
print(mypath + '/'+'Leszek_Miller')
print(type1_dict)

In [ ]:
print(len(type1_dict))
del_list = []
for names in type1_dict.keys():
    if len(type1_dict[names]) < threshold:
        del_list.append(names)
for name in del_list:
    del type1_dict[name]
print(len(del_list))

In [ ]:
type2_val = defaultdict(list)
type2_test = defaultdict(list)

for name in type2_dict.keys():
    size = len(type2_dict[name])
    if len(type2_dict[name]) >= 2:
        type2_val[name].extend(type2_dict[name][:size//2]) # 7  -  0, 3
        type2_test[name].extend(type2_dict[name][size//2 : size - size%2]) # 3, 6
    if size % 2:
        if np.random.rand() < 0.5:
            type2_val[name].append(type2_dict[name][-1])
        else:
            type2_test[name].append(type2_dict[name][-1])
# print(type2_dict)
# print(type2_val)
# print(type2_test)

In [ ]:
train = defaultdict(list)
type1_val = defaultdict(list)
type1_test = defaultdict(list)

for name in type1_dict.keys():
    size = len(type1_dict[name])
    #if size > imagesplit_thr:
    train[name] = type1_dict[name][:int(size*0.8)]
    size2 = size - int(size*0.8)
    #for image in type1_dict[name][size*0.8:]:
    if size2 >= 2:
        type1_val[name].extend(type1_dict[name][int(size*0.8):int(size*0.8)+size2//2]) 
        type1_test[name].extend(type1_dict[name][int(size*0.8)+size2//2 : size - size2%2])
    if size2 % 2:
        if np.random.rand() < 0.5:
            type1_val[name].append(type1_dict[name][-1])
        else:
            type1_test[name].append(type1_dict[name][-1])


In [ ]:
person_mapping = {}
i = 0
for path in train:
    person_mapping[path.split('/')[-1]] = i
    i+=1
for path in type2_val:
    person_mapping[path.split('/')[-1]] = i
    i+=1
for path in type2_test:
    person_mapping[path.split('/')[-1]] = i
    i+=1


In [ ]:
def get_data(dataset):
    data = []
    for path in dataset:
        for image in dataset[path]:
            data.append([path+'/'+image,person_mapping[path.split('/')[-1]]])
    return data

In [ ]:
data_train_todataset = get_data(train)
print(len(data_train_todataset))

data_val1_todataset = get_data(type1_val)
print(len(data_val1_todataset))

data_val2_todataset = get_data(type2_val)
print(len(data_val2_todataset))

data_test1_todataset = get_data(type1_test)
print(len(data_test1_todataset))

data_test2_todataset = get_data(type2_test)
print(len(data_test2_todataset))

In [ ]:
def read_image(path):
    return torchvision.io.read_image(path)
image = read_image('/content/drive/MyDrive/DLFAA/Final_Project/lfw/Ahmed_Chalabi/Ahmed_Chalabi_0001.jpg')

In [ ]:
transform = transforms.Compose([transforms.RandomHorizontalFlip(p=0.5),
                                             transforms.RandomVerticalFlip(p=0.5),
                                             transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [ ]:
img1 = read_image('/content/drive/MyDrive/DLFAA/Final_Project/lfw/Ahmed_Chalabi/Ahmed_Chalabi_0001.jpg')
img2 = read_image('/content/drive/MyDrive/DLFAA/Final_Project/lfw/Ahmed_Chalabi/Ahmed_Chalabi_0001.jpg')
concat = torch.cat([img1,img2],dim = 1)
print(concat.shape)

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data, data_dict, transform):
        self.data = data
        self.data_dict = data_dict
        self.length = len(data)
        self.transform_flag = transform
        self.transform = transforms.Compose([transforms.RandomHorizontalFlip(p=0.5), 
                                             transforms.RandomVerticalFlip(p=0.5)
                                             ]) ###is this helpful
                                            #  transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]) # transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
                                             ###add more tranformations
        self.normalize = transforms.Compose([transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, index):

        anchor = read_image(self.data[index][0])
        anchor = anchor.float()/255
        negative = random.choice(range(self.length))
        while self.data[negative][1] == self.data[index][1]:
            negative = random.choice(range(self.length))
        negative = read_image(self.data[negative][0])
        negative = negative.float()/255

        positive_key = '/'.join(self.data[index][0].split('/')[:-1])
        positive = random.choice(self.data_dict[positive_key])
        positive = read_image(positive_key + '/' + positive)
        positive = positive.float()/255

        if self.transform_flag:
            anchor = self.transform(anchor) ###may try removing transform for anchor and negative and try
            negative = self.transform(negative)
            positive = self.transform(positive)
        anchor = self.normalize(anchor) 
        negative = self.normalize(negative)
        positive = self.normalize(positive)
        pos_example = torch.cat([anchor,positive],dim=1)
        neg_example = torch.cat([anchor,negative],dim=1)
        return torch.stack([pos_example,neg_example],dim=3)
        

In [ ]:
def collate_fn(data):
    data = torch.stack(data)
    length = data.shape[0]
    data = torch.cat([data[:,:,:,:,0],data[:,:,:,:,1]])
    labels = torch.cat([torch.ones(length),torch.zeros(length)])
    idx = torch.randperm(2*length)
    data = data[idx]
    labels = labels[idx]
    return data, labels

training_data = DataLoader(Dataset(data_train_todataset, train, True), 
                           params['batchsize'], 
                           params['shuffle'], 
                           collate_fn = collate_fn, 
                           num_workers=params['num_workers'])

val1_data = DataLoader(Dataset(data_val1_todataset, type1_val, False), 
                           params['batchsize'], 
                           params['shuffle'], 
                           collate_fn = collate_fn, 
                           num_workers=params['num_workers'])

val2_data = DataLoader(Dataset(data_val2_todataset, type2_val, False), 
                           params['batchsize'], 
                           params['shuffle'], 
                           collate_fn = collate_fn, 
                           num_workers=params['num_workers'])

test1_data = DataLoader(Dataset(data_test1_todataset, type1_test, False), 
                           params['batchsize'], 
                           params['shuffle'], 
                           collate_fn = collate_fn, 
                           num_workers=params['num_workers'])

test2_data = DataLoader(Dataset(data_test2_todataset, type2_test, False), 
                           params['batchsize'], 
                           params['shuffle'], 
                           collate_fn = collate_fn, 
                           num_workers=params['num_workers'])

In [ ]:
"""for data,label in training_data:
    print(data.shape)"""

In [ ]:
model_ft = models.resnet18(pretrained=True)
# num_ftrs = model_ft.fc.in_features
# model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = nn.Sequential(
    model_ft,
    nn.ReLU(),
    nn.Linear(1000, 2)
)

model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_ft.parameters(), params['lr'])
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [ ]:
#summary(model_ft,(3,500,250))
torch.cuda.empty_cache()

In [ ]:
results_loss = defaultdict()
results_acc = defaultdict()
results_loss['train'] = []
results_acc['train'] = []

results_loss['val1'] = []
results_acc['val1'] = []

results_loss['val2'] = []
results_acc['val2'] = []

for epoch in range(params['epochs']):
    model_ft.train()
    running_loss = 0.0
    batch_count = 0
    correct = 0
    total = 0
    for i, x in enumerate(training_data, 0):
        batch_count += 1
        inputs, labels = x
        labels = labels.type(torch.LongTensor)
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model_ft(inputs)
        loss = criterion(outputs,labels)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # print("Epoch", epoch+1, "Batch:", batch_count, "Loss", loss.item(), "accuracy", (predicted == labels).sum().item()/labels.size(0))

        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print('Accuracy of train images: {}'.format(correct / total))
    print("E:{}, Train Loss:{}".format(epoch+1, running_loss / batch_count))

    results_loss['train'].append(running_loss / batch_count)
    results_acc['train'].append(correct / total)

    #validation1
    model_ft.eval()
    correct = 0
    total = 0
    val_loss = 0.0
    batch_count = 0
    with torch.no_grad():
        for data in val1_data:

            batch_count+=1
            images,labels = data
            labels=labels.type(torch.LongTensor)
            images = images.to(device)
            labels = labels.to(device)
            outputs = model_ft(images)  
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
                
            loss = criterion(outputs, labels)       
            val_loss += loss.item()
    val_loss /= batch_count
    print('Epoch {} Accuracy of val1 images: {}'.format(epoch+1,  correct / total))
    print('Epoch {} Val1 Loss: {}'.format(epoch+1, val_loss))

    results_loss['val1'].append(val_loss)
    results_acc['val1'].append(correct / total)

    #validation2
    model_ft.eval()
    correct = 0
    total = 0
    val_loss = 0.0
    batch_count = 0
    with torch.no_grad():
        for data in val2_data:

            batch_count+=1
            images, labels = data
            labels = labels.type(torch.LongTensor)
            images = images.to(device)
            labels = labels.to(device)
            outputs = model_ft(images)  
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
                
            loss = criterion(outputs,labels)       
            val_loss += loss.item()
    val_loss /= batch_count
    print('Epoch {} Accuracy of val2 images: {}'.format(epoch+1,  correct / total))
    print('Epoch {} Val2 Loss: {}'.format(epoch+1, val_loss))

    results_loss['val2'].append(val_loss)
    results_acc['val2'].append(correct / total)

In [ ]:
print(results_loss)
print(results_acc)

In [ ]:
with open('/content/drive/MyDrive/DLFAA/Final_Project/base_results.txt','a') as data: 
    data.write('Loss\n')
    data.write(str(results_loss))
    data.write('\nAccuracy\n')
    data.write(str(results_acc))

In [ ]:
model_ft.eval()
correct = 0
total = 0
val_loss = 0.0
batch_count = 0
with torch.no_grad():
    for data in test1_data:
        batch_count+=1
        images, labels = data
        labels = labels.type(torch.LongTensor)
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_ft(images)  
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item() 
        loss = criterion(outputs,labels)       
        val_loss += loss.item()
val_loss /= batch_count
print('Accuracy of test1 images: {}'.format(  correct / total))
print('test1 Loss: {}'.format( val_loss))